In [19]:
import os
import requests
import zipfile
import subprocess
import shutil
from rdflib import Graph

# Tải về công cụ LUBM UBA
def download_lubm_uba():
    url = "http://swat.cse.lehigh.edu/projects/lubm/uba1.7.zip"
    zip_file_path = "LUBM_Data/uba1.7.zip"
    extract_folder = "LUBM_Data/uba"

    # Tạo thư mục cho dự án
    os.makedirs("LUBM_Data", exist_ok=True)

    # Tải về tệp zip
    print("Đang tải công cụ LUBM UBA...")
    response = requests.get(url)
    if response.status_code == 200:
        with open(zip_file_path, "wb") as f:
            f.write(response.content)
        print("Tải về thành công.")
    else:
        print("Có lỗi xảy ra khi tải tệp.")
        return False

    # Giải nén tệp zip
    print("Đang giải nén tệp...")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
    print("Giải nén thành công.")

    return True

# Tạo dữ liệu OWL sử dụng LUBM UBA
def generate_lubm_data(num_universities=5):
    # Đường dẫn tới tệp uba.jar
    uba_jar_path = os.path.abspath(r"LUBM_Data/uba/classes/uba.jar")
    # Đường dẫn tới tệp ontology, thay đổi để có định dạng phù hợp
    ontology_path = os.path.abspath(r"LUBM_Data/uba/univ-bench.owl").replace("\\", "/")

    # Kiểm tra xem tệp JAR có tồn tại tại đường dẫn này hay không
    if not os.path.exists(uba_jar_path):
        print(f"Lỗi: Không tìm thấy tệp JAR tại đường dẫn {uba_jar_path}")
        return

    # Kiểm tra xem tệp ontology có tồn tại hay không
    if not os.path.exists(ontology_path):
        print(f"Lỗi: Không tìm thấy tệp ontology tại đường dẫn {ontology_path}")
        return

    # Câu lệnh để chạy uba.jar với số lượng trường đại học mong muốn và chỉ định ontology
    command = f'java -jar "{uba_jar_path}" -univ {num_universities} -onto file:"{ontology_path}"'
    try:
        print(f"Đang tạo dữ liệu OWL cho {num_universities} trường đại học...")
        result = subprocess.run(command, check=True, shell=True)
        print("Dữ liệu OWL đã được tạo thành công.")
    except subprocess.CalledProcessError as e:
        print(f"Lỗi khi tạo dữ liệu LUBM: {e}")

# Di chuyển các tệp OWL đến thư mục đầu ra
def move_owl_files_to_output():
    current_directory = os.getcwd()
    output_directory = os.path.abspath(r"LUBM_Data/data")

    # Tạo thư mục đầu ra nếu nó chưa tồn tại
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Tìm và di chuyển các tệp OWL
    for file_name in os.listdir(current_directory):
        if file_name.endswith(".owl"):
            source_path = os.path.join(current_directory, file_name)
            destination_path = os.path.join(output_directory, file_name)
            shutil.move(source_path, destination_path)
            print(f"Đã di chuyển {file_name} đến {output_directory}")

# Chuyển đổi OWL sang RDF
def convert_owl_to_rdf(input_directory, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for file_name in os.listdir(input_directory):
        if file_name.endswith(".owl"):
            input_path = os.path.join(input_directory, file_name)
            output_path = os.path.join(output_directory, file_name.replace(".owl", ".rdf"))
            g = Graph()
            # Sử dụng URI hợp lệ cho ontology
            g.parse(input_path, format='xml')
            g.serialize(destination=output_path, format='xml')
            print(f"Đã chuyển đổi {file_name} thành {output_path}")

# Thực hiện tải và tạo dữ liệu LUBM
if download_lubm_uba():
    generate_lubm_data(num_universities=5)
    # Di chuyển các tệp OWL đến thư mục đầu ra
    move_owl_files_to_output()
    # Chuyển đổi OWL sang RDF
    convert_owl_to_rdf("LUBM_Data/data", "LUBM_Data/data")


Đang tải công cụ LUBM UBA...
Tải về thành công.
Đang giải nén tệp...
Giải nén thành công.
Đang tạo dữ liệu OWL cho 5 trường đại học...
Dữ liệu OWL đã được tạo thành công.
Đã di chuyển University0_0.owl đến c:\Users\DELL\Desktop\Linh\CaoHoc\T6.GiaiThuat\Code\data\LUBM_Data\data
Đã di chuyển University0_1.owl đến c:\Users\DELL\Desktop\Linh\CaoHoc\T6.GiaiThuat\Code\data\LUBM_Data\data
Đã di chuyển University0_10.owl đến c:\Users\DELL\Desktop\Linh\CaoHoc\T6.GiaiThuat\Code\data\LUBM_Data\data
Đã di chuyển University0_11.owl đến c:\Users\DELL\Desktop\Linh\CaoHoc\T6.GiaiThuat\Code\data\LUBM_Data\data
Đã di chuyển University0_12.owl đến c:\Users\DELL\Desktop\Linh\CaoHoc\T6.GiaiThuat\Code\data\LUBM_Data\data
Đã di chuyển University0_13.owl đến c:\Users\DELL\Desktop\Linh\CaoHoc\T6.GiaiThuat\Code\data\LUBM_Data\data
Đã di chuyển University0_14.owl đến c:\Users\DELL\Desktop\Linh\CaoHoc\T6.GiaiThuat\Code\data\LUBM_Data\data
Đã di chuyển University0_2.owl đến c:\Users\DELL\Desktop\Linh\CaoHoc\T6.Gia

In [21]:
import os

def correct_ontology_paths(input_directory):
    for file_name in os.listdir(input_directory):
        if file_name.endswith(".rdf"):
            file_path = os.path.join(input_directory, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()

            # Thay thế đường dẫn trùng lặp không hợp lệ bằng URI đúng chuẩn
            corrected_content = content.replace(
                "file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/data/c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#",
                "file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#"
            )

            # Ghi lại nội dung đã sửa vào tệp gốc
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(corrected_content)

            print(f"Đã sửa đường dẫn trong tệp {file_name}")

# Thực hiện sửa các tệp OWL trong thư mục đầu ra
correct_ontology_paths("LUBM_Data/data")


Đã sửa đường dẫn trong tệp University0_0.rdf
Đã sửa đường dẫn trong tệp University0_1.rdf
Đã sửa đường dẫn trong tệp University0_10.rdf
Đã sửa đường dẫn trong tệp University0_11.rdf
Đã sửa đường dẫn trong tệp University0_12.rdf
Đã sửa đường dẫn trong tệp University0_13.rdf
Đã sửa đường dẫn trong tệp University0_14.rdf
Đã sửa đường dẫn trong tệp University0_2.rdf
Đã sửa đường dẫn trong tệp University0_3.rdf
Đã sửa đường dẫn trong tệp University0_4.rdf
Đã sửa đường dẫn trong tệp University0_5.rdf
Đã sửa đường dẫn trong tệp University0_6.rdf
Đã sửa đường dẫn trong tệp University0_7.rdf
Đã sửa đường dẫn trong tệp University0_8.rdf
Đã sửa đường dẫn trong tệp University0_9.rdf
Đã sửa đường dẫn trong tệp University1_0.rdf
Đã sửa đường dẫn trong tệp University1_1.rdf
Đã sửa đường dẫn trong tệp University1_10.rdf
Đã sửa đường dẫn trong tệp University1_11.rdf
Đã sửa đường dẫn trong tệp University1_12.rdf
Đã sửa đường dẫn trong tệp University1_13.rdf
Đã sửa đường dẫn trong tệp University1_14.rdf
